In [1]:
import torch
import torch.nn as nn
from torch.nn import Parameter
from torch.autograd import Variable
from torch import optim
import torch.nn.functional as F
use_cuda = torch.cuda.is_available()
torch.cuda.set_device(0)
import sys, random
import numpy as np
try:
    import cPickle as pickle
except:
    import pickle
import math
from sklearn.metrics import roc_auc_score

In [2]:
train_sl= pickle.load(open('/data/projects/py_ehr_2/Data/hf50_cl2_h143_ref_t.train', 'rb'), encoding='bytes')
test_sl= pickle.load(open('/data/projects/py_ehr_2/Data/hf50_cl2_h143_ref_t.test', 'rb'), encoding='bytes')
valid_sl= pickle.load(open('/data/projects/py_ehr_2/Data/hf50_cl2_h143_ref_t.valid', 'rb'), encoding='bytes')
print (len(train_sl),len(valid_sl),len(test_sl))

29504 4269 8956


In [3]:
'''
# For debugging

train_sl= [[12849431,  0 , 
  [[[0], [8, 6, 7]],
   [[77], [7, 6, 5, 9]],
   [[2], [1, 6, 9]]]],
 [12982169,
  0,
  [[[0], [2, 7, 8, 1]],
   [[7], [4, 1, 3, 8]]]],
 [12982169,1,
  [[[0], [3, 8, 1,9]],
   [[14], [3, 2, 6, 8]]]],
 [12785132,
  1,
  [[[0], [3, 8, 1]], 
   [[10], [3, 8, 1,9]],
   [[70], [3, 8, 7, 6, 9]],
   [[156], [3, 6, 4]]]]]

valid_sl= [[12849431,  0 , 
  [[[0], [8, 6, 7]],
   [[2], [1, 6, 9]]]],
 [12785132,
  1,
  [[[0], [3, 8, 1]], 
   [[156], [3, 6, 4]]]]]

test_sl= [[12982169,
  0,
  [[[0], [2, 7 ]],
   [[7], [4, 1, 8]]]],
 [12982169,1,
  [[[0], [3, 8]],
   [[14], [3, 2]]]]]
'''

'\n# For debugging\n\ntrain_sl= [[12849431,  0 , \n  [[[0], [8, 6, 7]],\n   [[77], [7, 6, 5, 9]],\n   [[2], [1, 6, 9]]]],\n [12982169,\n  0,\n  [[[0], [2, 7, 8, 1]],\n   [[7], [4, 1, 3, 8]]]],\n [12982169,1,\n  [[[0], [3, 8, 1,9]],\n   [[14], [3, 2, 6, 8]]]],\n [12785132,\n  1,\n  [[[0], [3, 8, 1]], \n   [[10], [3, 8, 1,9]],\n   [[70], [3, 8, 7, 6, 9]],\n   [[156], [3, 6, 4]]]]]\n\nvalid_sl= [[12849431,  0 , \n  [[[0], [8, 6, 7]],\n   [[2], [1, 6, 9]]]],\n [12785132,\n  1,\n  [[[0], [3, 8, 1]], \n   [[156], [3, 6, 4]]]]]\n\ntest_sl= [[12982169,\n  0,\n  [[[0], [2, 7 ]],\n   [[7], [4, 1, 8]]]],\n [12982169,1,\n  [[[0], [3, 8]],\n   [[14], [3, 2]]]]]\n'

In [3]:
class EHR_RNN(nn.Module):
    def __init__(self, input_size,embed_dim, hidden_size, n_layers=1,dropout_r=0.1,cell_type='LSTM',bi=False ,time=False, preTrainEmb=''):
        super(EHR_RNN, self).__init__()
        self.n_layers = n_layers
        self.hidden_size = hidden_size
        self.embed_dim = embed_dim
        self.dropout_r = dropout_r
        self.cell_type = cell_type
        self.preTrainEmb=preTrainEmb
        self.time=time
        
        if bi: self.bi=2 
        else: self.bi=1
        
        if self.time: self.in_size= embed_dim+1 
        else: self.in_size=embed_dim
            
        if len(self.preTrainEmb)>0:
            emb_t= torch.FloatTensor(np.asmatrix(self.preTrainEmb))
            self.embed= nn.Embedding.from_pretrained(emb_t)#,freeze=False)  
        else:
            self.embed= nn.Embedding(input_size, self.embed_dim,padding_idx=0)#,scale_grad_by_freq=True)
        
        if self.cell_type == "GRU":
            cell = nn.GRU
        elif self.cell_type == "RNN":
            cell = nn.RNN
        elif self.cell_type == "LSTM":
            cell = nn.LSTM
        else:
            raise NotImplementedError
            
          
        self.rnn_c = cell(self.in_size, hidden_size,num_layers=n_layers, dropout= dropout_r , bidirectional=bi , batch_first=True)
        self.out = nn.Linear(self.hidden_size*self.bi,1)
        self.sigmoid = nn.Sigmoid()

        
    def EmbedPatient_MB(self, input): # x is a ehr_seq_tensor

        if use_cuda:
            flt_typ=torch.cuda.FloatTensor
            lnt_typ=torch.cuda.LongTensor
        else: 
            lnt_typ=torch.LongTensor
            flt_typ=torch.FloatTensor
        mb=[]
        mtd=[]
        lbt=[]
        seq_l=[]
        self.bsize=len(input) ## no of pts in minibatch
        lp= len(max(input, key=lambda xmb: len(xmb[-1]))[-1]) ## maximum number of visits per patients in minibatch
        llv=0
        for x in input:
            lv= len(max(x[-1], key=lambda xmb: len(xmb[1]))[1])
            if llv < lv:
                llv=lv     # max number of codes per visit in minibatch        

        for pt in input:
            sk,label,ehr_seq_l = pt
            lpx=len(ehr_seq_l) ## no of visits in pt record
            seq_l.append(lpx) 
            lbt.append(Variable(flt_typ([[float(label)]])))### check if code issue replace back to the above
            
            ml=(len(max(ehr_seq_l, key=len))) # max number of codes per visit in pt record
            ehr_seq_tl=[]
            time_dim=[]
            for ehr_seq in ehr_seq_l:
                pd=(0, (llv -len(ehr_seq[1])))
                result = F.pad(torch.from_numpy(np.asarray(ehr_seq[1],dtype=int)).type(lnt_typ),pd,"constant", 0)
                ehr_seq_tl.append(result)
                if self.time:                 
                    time_dim.append(Variable(torch.from_numpy(np.asarray(ehr_seq[0],dtype=int)).type(flt_typ)))
            ehr_seq_t= Variable(torch.stack(ehr_seq_tl,0)) 
            lpp= lp-lpx ## diff be max seq in minibatch and cnt of pt visits
            zp= nn.ZeroPad2d((0,0,lpp,0)) ## (0,0,0,lpp) when use the pack padded seq and (0,0,lpp,0) otherwise
            ehr_seq_t= zp(ehr_seq_t) ## zero pad the visits med codes
            mb.append(ehr_seq_t)
            if self.time:
                time_dim_v= Variable(torch.stack(time_dim,0))
                time_dim_pv= zp(time_dim_v)## zero pad the visits time diff codes
                mtd.append(time_dim_pv)

            
        mb_t= Variable(torch.stack(mb,0)) 
        if use_cuda:
            mb_t.cuda()
        embedded = self.embed(mb_t)  ## Embedding for codes
        embedded = torch.sum(embedded, dim=2) 
        lbt_t= Variable(torch.stack(lbt,0))
        if self.time:
            mtd_t= Variable(torch.stack(mtd,0))
            if use_cuda: mtd_t.cuda()
            out_emb= torch.cat((embedded,mtd_t),dim=2)
        else:
            out_emb= embedded
        return out_emb, lbt_t,seq_l #,dem_emb
    
    def init_hidden(self):
        
        h_0 = Variable(torch.rand(self.n_layers*self.bi,self.bsize, self.hidden_size))
        
        if self.cell_type == "LSTM":
            result = (h_0,h_0)
        else: 
            result = h_0
            
        if use_cuda:
            return result.cuda()
        else:
            return result
    
    def forward(self, input):
        
        x_in , lt ,x_lens = self.EmbedPatient_MB(input)
        #x_inp = nn.utils.rnn.pack_padded_sequence(x_in,x_lens,batch_first=True)   
        #h_0= self.init_hidden()
        output, hidden = self.rnn_c(x_in)#,h_0) 
        if self.cell_type == "LSTM":
            hidden=hidden[0]
        if self.bi==2:
            output = self.sigmoid(self.out(torch.cat((hidden[-2],hidden[-1]),1)))
        else:
            output = self.sigmoid(self.out(hidden[-1]))
        return output.squeeze(), lt.squeeze()


In [4]:
def train (tmodel,mini_batch, criterion, optimizer):  
    tmodel.zero_grad()
    output , label_tensor = tmodel(mini_batch)
    loss = criterion(output, label_tensor)
    loss.backward()
    optimizer.step()
   
    return output, loss.item()

In [5]:
def run_model_train(tmodel,dataset,batch_size,optimizer):
        
    tmodel.train()
    dataset.sort(key=lambda pt:len(pt[-1]),reverse=True) 
    # Keep track of losses for plotting
    current_loss = 0
    all_losses = []
    print_every = 10#int(batch_size/2)
    plot_every = 5
    iter=0
    n_batches = int(np.ceil(int(len(dataset)) / int(batch_size)))
    start = time.time()

    for index in random.sample(range(n_batches), n_batches):
            batch = dataset[index*batch_size:(index+1)*batch_size]
            output, loss = train(tmodel,batch, criterion = nn.BCELoss(), optimizer = optimizer)
            current_loss += loss
            iter +=1
            # Add current loss avg to list of losses
            if iter % plot_every == 0:
                all_losses.append(current_loss / plot_every)
                current_loss = 0
                
    return current_loss,all_losses


In [6]:
def calculate_auc(test_model, dataset, batch_size=200):
    test_model.eval()
    dataset.sort(key=lambda pt:len(pt[-1]),reverse=True) 
    n_batches = int(np.ceil(int(len(dataset)) / int(batch_size)))
    labelVec =[]
    y_hat= []
    
    for index in range(n_batches):
            batch = dataset[index*batch_size:(index+1)*batch_size]
            output, label_t = test_model(batch)
            y_hat.extend(output.cpu().data.view(-1).numpy())
            labelVec.extend(label_t.cpu().data.view(-1).numpy())
    auc = roc_auc_score(labelVec, y_hat)
    
    return auc

In [7]:
# training all samples in random order
import time
import math

def timeSince(since):
    now = time.time()
    s = now - since
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

In [ ]:
epochs=100
batch_size=100
## Optim param
learning_rate = 0.01
l2=1e-04
epsl=1e-06 
## for final results reporting
current_loss_l=[]
all_losses_l=[]
train_auc_allep =[]
valid_auc_allep =[]
test_auc_allep=[]
bestValidAuc = 0.0
bestTestAuc = 0.0
bestValidEpoch = 0

### Define Model Parameters
model = EHR_RNN(input_size=15816, hidden_size=64 ,embed_dim=128, dropout_r=0, cell_type='GRU', n_layers=1,bi=False ,time=True, preTrainEmb='')
if use_cuda:
    model = model.cuda()
    
### Select Optimizer
####################
#optimizer = optim.SGD(tmodel.parameters(), lr=learning_rate)#, weight_decay=l2)
#optimizer = optim.Adadelta(tmodel.parameters(), lr=learning_rate, weight_decay=l2)
#optimizer = optim.ASGD(tmodel.parameters(), lr=learning_rate, weight_decay=l2 )
#optimizer = optim.SparseAdam (tmodel.parameters(),lr=learning_rate) #'''lr=learning_rate,''' 
#optimizer = optim.Adagrad (tmodel.parameters(),lr=learning_rate, weight_decay=l2) #'''lr=learning_rate,''' 
optimizer = optim.Adamax(model.parameters(), lr=learning_rate, weight_decay=l2 ,eps=epsl)
#optimizer = optim.Adamax(filter(lambda p: p.requires_grad, tmodel.parameters()), lr=learning_rate, weight_decay=l2 ,eps=epsl) ### Beta defaults (0.9, 0.999)
#optimizer = optim.RMSprop (tmodel.parameters(),lr=learning_rate, weight_decay=l2 ,eps=epsl)
#optimizer = optim.Adam(tmodel.parameters(), lr=learning_rate, weight_decay=learning_rate)
    
### Run Epochs    
for ep in range(epochs):
    
    #print (model.embed.weight.data[135] ) ## checkpoint  for embedding
    #print (model.state_dict() )## checkpoint  for all learnable parameters
    start = time.time()
    current_loss_la,all_losses_la = run_model_train(model,train_sl,batch_size,optimizer)
    train_time = timeSince(start)
    eval_start = time.time()
    train_auc = calculate_auc(model,train_sl,batch_size)
    test_auc = calculate_auc(model,test_sl,batch_size)
    valid_auc = calculate_auc(model,valid_sl,batch_size)
    eval_time = timeSince(eval_start)
    all_losses_l.append (all_losses_la)
    avg_loss = np.mean(all_losses_la)
    train_auc_allep.append(train_auc)
    valid_auc_allep.append(valid_auc)
    test_auc_allep.append(test_auc)
    current_loss_l.append(current_loss_la)
    print ("Epoch ", ep," Train_auc :", train_auc, " , Valid_auc : ", valid_auc, " ,& Test_auc : " , test_auc," Avg Loss: ", avg_loss, 'Train Time (%s) Eval Time (%s)'%(train_time,eval_time) )
     
    if valid_auc > bestValidAuc: 
        bestValidAuc = valid_auc
        bestValidEpoch = ep
        bestTestAuc = test_auc
        best_model = model
        #torch.save(best_model, bmodel_pth)
        #torch.save(best_model.state_dict(), bmodel_st)
    if ep - bestValidEpoch >12: break
            
print ('bestValidAuc %f has a TestAuc of %f at epoch %d ' % (bestValidAuc, bestTestAuc, bestValidEpoch))

# Baseline LSTM Epoch  5  Train_auc : 0.8761979004359334  , Valid_auc :  0.8298963079092181  ,& Test_auc :  0.8381190037440037  Avg Loss:  0.26761469113624703 Train Time (1m 3s) Eval Time (1m 30s)
# Baseline LSTM with time Epoch  7  Train_auc : 0.885175808581514  , Valid_auc :  0.8270585852421586  ,& Test_auc :  0.8349125911625912  Avg Loss:  0.2580455846453117 Train Time (1m 29s) Eval Time (2m 7s)
# Baseline GRU with time 

###Older results for ref
# GRU: new data format : bestValidAuc 0.831529 has a TestAuc of 0.833067 at epoch 16    
## versus bestValidAuc 0.821507 has a TestAuc of 0.837944 at epoch 2 with default h_0
# h_0 random

Epoch  0  Train_auc : 0.8156365665816847  , Valid_auc :  0.7966185646194801  ,& Test_auc :  0.8026395713895714  Avg Loss:  0.32602816749427277 Train Time (0m 57s) Eval Time (1m 20s)
Epoch  1  Train_auc : 0.8481077178242497  , Valid_auc :  0.8229945497206469  ,& Test_auc :  0.8296466718341718  Avg Loss:  0.291267538474778 Train Time (0m 56s) Eval Time (1m 20s)
Epoch  2  Train_auc : 0.8622246877789356  , Valid_auc :  0.8237600088382908  ,& Test_auc :  0.8354166666666667  Avg Loss:  0.274739132341692 Train Time (0m 56s) Eval Time (1m 20s)
